In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import talib
import pandas as pd
import pandas_ta as ta
from stock_env.feature.feature_extractor import *
from stock_env.utils import check_col
import mt4_hst

In [2]:
# note that all ndarrays must be the same length!
df = mt4_hst.read_hst("../stock_datasets/" + 'FPT' + "1440.hst")

In [9]:
def func1(df):
    t = ta.Strategy(
        name="Trend Signal Strategy",
        description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
        ta=[
            {"kind": "sma", "length": 50},
            {"kind": "ema", "length": 10},
            {"kind": "ema", "length": 20},])
    return df.ta.strategy(t)

def func2(df):
    df['SMA_50'] = ta.sma(close=df['close'], length=50)
    df['EMA_10'] = ta.ema(close=df['close'], length=10)
    df['EMA_20'] = ta.ema(close=df['close'], length=20)
    return df

def func3(df):
    df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)
    df['EMA_10'] = talib.EMA(df['close'], timeperiod=10)
    df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)
    return df

def func4(df):
    df.ta.sma(length=50, append=True)
    df.ta.ema(length=10, append=True)
    df.ta.ema(length=20, append=True)
    df['LOW_10'] = talib.MIN(df.low, timeperiod=10)
    df['HIGH_10'] = talib.MAX(df.high, timeperiod=10)
    df['LOW_50'] = talib.MIN(df.low, timeperiod=50)
    df['VOLUME_SMA_20'] = talib.SMA(df.volume, timeperiod=20)
    df.ta.adx(length=20, scalar=1, append=True)
    df.ta.aroon(length=20, scalar=1, talib=False, append=True)
    df.ta.stc(tclength=10, ma1=df['EMA_10'], ma2=df['EMA_20'], fast=10, slow=20, append=True)
    df.ta.natr(length=20, scalar=1, talib=False, append=True)
    df.ta.rsi(length=20, scalar=1, talib=False, append=True)
    df.ta.cci(length=20, scalar=1, append=True)
    df['CCI_20_0.015'] = df['CCI_20_0.015'] / 100
    df[['STC_10_10_20_0.5', 'STCstoch_10_10_20_0.5']] = df[['STC_10_10_20_0.5', 'STCstoch_10_10_20_0.5']] / 100
    df['EMA_ratio'] = df['EMA_10'] / df['EMA_20']
    df['LOW_ratio'] = df['LOW_10'] / df['LOW_50']
    df['above_MA'] = (df['close'] > df['SMA_50']).astype(int)
    return df

In [6]:
import time
s = time.time()
_df1 = TrendFeatures().preprocess(df.copy())
e = time.time()
t1 = e-s
print(t1)

s = time.time()
_df2 = TrendFeaturesV2().preprocess(df.copy())
e = time.time()
t2 = e-s
print(t2)

2.745360851287842
0.3604402542114258


In [2]:
from stock_env.data_loader import RandomStockLoader
from stock_env.feature import TrendFeaturesV2

data_loader = RandomStockLoader(
    tickers = "SSI".split(),
    data_folder_path = "../stock_datasets/",
    feature_extractor = TrendFeaturesV2
)

In [4]:
obs = data_loader.reset()
ohlcv = data_loader.current_ohlcv

In [7]:
ohlcv.open.item()

5.158303737640381